In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

## Task 1 Transfer Learning 

The section below contains  the core utility functions

In [2]:
# ========================
# Dataset preparation
# ========================
class RetinaMultiLabelDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.image_dir, row.iloc[0])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row[1:].values.astype("float32"))
        if self.transform:
            img = self.transform(img)
        return img, labels

class RetinaTestDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        df = pd.read_csv(csv_file)
        self.ids = df.iloc[:, 0].values  # first column -> id/ID
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_path = os.path.join(self.image_dir, img_id)
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_id


In [3]:
# ========================
# build model
# ========================
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

def build_model(backbone="resnet18", num_classes=3, pretrained=True):
    if backbone == "resnet18":
        weights = ResNet18_Weights.IMAGENET1K_V1 if pretrained else None
        model = resnet18(weights=weights)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

    elif backbone == "efficientnet":
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        model = efficientnet_b0(weights=weights)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

    else:
        raise ValueError("Unsupported backbone")

    return model


In [4]:
# ========================
# model training and val
# ========================
def train_one_backbone(
    backbone,
    train_csv,
    val_csv,
    test_csv,
    train_image_dir,
    val_image_dir,
    test_image_dir,
    epochs=10,
    batch_size=32,
    lr=1e-4,
    img_size=256,
    save_dir="checkpoints",
    pretrained_backbone=None,
    task="full_ft",
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    task_name_map = {
        "no_finetune": "Task1.1 No fine-tuning",
        "cls_only": "Task1.2 Frozen backbone, classifier only",
        "full_ft": "Task1.3 Full fine-tuning",
    }

    print("===========================================")
    print(f"{task_name_map[task]}  |  Backbone: {backbone}")
    print("===========================================")

    # transforms
    transform = transforms.Compose(
        [
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    )

    # datasets & dataloaders
    train_ds = RetinaMultiLabelDataset(train_csv, train_image_dir, transform)
    val_ds = RetinaMultiLabelDataset(val_csv, val_image_dir,transform)
    test_ds = RetinaMultiLabelDataset(test_csv, test_image_dir, transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=0)
    
    # compute pos_weight
    label_cols = train_ds.data.columns[1:]  # skip ID
    pos_counts = train_ds.data[label_cols].sum(axis=0).values.astype(np.float32)
    neg_counts = len(train_ds) - pos_counts
    pos_weight_np = neg_counts / (pos_counts + 1e-6)  # avoid division by zero

    pos_weight = torch.tensor(pos_weight_np, dtype=torch.float32, device=device)
    print("pos_weight:", pos_weight)


    # model
    model = build_model(backbone, num_classes=3, pretrained=False).to(device)

    if pretrained_backbone is not None:
        state_dict = torch.load(pretrained_backbone, map_location="cpu")
        model.load_state_dict(state_dict)
        print(f"Loaded pretrained weights from {pretrained_backbone}")

    # set which parameters are trainable
    if task == "no_finetune":
        # everything is frozen
        for p in model.parameters():
            p.requires_grad = False
        optimizer = None
    elif task == "cls_only":
        # freeze backbone
        for p in model.parameters():
            p.requires_grad = False
        # unfreeze classifier
        if backbone == "resnet18":
            for p in model.fc.parameters():
                p.requires_grad = True
        elif backbone == "efficientnet":
            for p in model.classifier.parameters():
                p.requires_grad = True
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    else:  # full_ft
        for p in model.parameters():
            p.requires_grad = True
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
        #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,)


    criterion = nn.BCEWithLogitsLoss(pos_weight = pos_weight)

    # checkpoint path (unique per backbone + task)
    os.makedirs(save_dir, exist_ok=True)
    task_file_map = {
    "no_finetune": "task1_1",
    "cls_only":    "task1_2",
    "full_ft":     "task1_3",
    }
    task_prefix = task_file_map[task] 
    ckpt_path = os.path.join(save_dir, f"csu_{task_prefix}_{backbone}.pt")

    # ========= TRAINING (only for Task1.2 and Task1.3) =========
    if task != "no_finetune":
        best_val_loss = float("inf")

        for epoch in range(epochs):
            model.train()
            train_loss = 0.0
            for imgs, labels in train_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item() * imgs.size(0)

            train_loss /= len(train_loader.dataset)

            # validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for imgs, labels in val_loader:
                    imgs, labels = imgs.to(device), labels.to(device)
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * imgs.size(0)
            val_loss /= len(val_loader.dataset)
            #scheduler.step(val_loss)

            print(f"[{backbone}] Epoch {epoch+1}/{epochs} Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f}")

            # save best
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), ckpt_path)
                print(f"Saved best model for {backbone} ({task}) at {ckpt_path}")
    else:
        # Task1.1: no fine-tuning – directly evaluate the pretrained model
        torch.save(model.state_dict(), ckpt_path)
        print(f"[{backbone}] {task_name_map[task]}: no training, model saved at {ckpt_path}")

    # ========= OFFSITE TEST EVALUATION =========
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    model.to(device)
    model.eval()

    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)
            y_true.extend(labels.numpy())
            y_pred.extend(preds)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    return ckpt_path, y_true, y_pred  # used later for Kaggle submission


In [5]:
# Function used to generate submission for kaggle
def generate_kaggle_submission(
    backbone,
    ckpt_path,
    onsite_csv,
    onsite_image_dir,
    img_size=256,
    batch_size=32,
    out_csv="submission.csv",
    threshold=0.5,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    transform = transforms.Compose(
        [
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
        ]
    )

    # build model and load weights
    model = build_model(backbone, num_classes=3, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    model.eval()

    # read the original Kaggle template
    template = pd.read_csv(onsite_csv)
    id_col_name = template.columns[0]  # should be 'id'

    # dataset and loader use the same csv for IDs
    test_ds = RetinaTestDataset(onsite_csv, onsite_image_dir, transform)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0)

    ids = []
    probs_all = []

    with torch.no_grad():
        for imgs, img_ids in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy() 
            ids.extend(img_ids)
            probs_all.append(probs)

    probs_all = np.concatenate(probs_all, axis=0)

    template_ids = template[id_col_name].values
    ids = np.array(ids)

    if not np.array_equal(template_ids, ids):
        print("WARNING: IDs in template and predictions do not match exactly in order!")
        id_to_idx = {image_id: i for i, image_id in enumerate(ids)}
        reorder_idx = [id_to_idx[x] for x in template_ids]
        probs_all = probs_all[reorder_idx, :]

    # convert probabilities to 0/1 labels using threshold
    bin_preds = (probs_all >= threshold).astype(int)  # int64 by default

    # overwrite D/G/A in template; keeps int dtypes and exact structure
    template["D"] = bin_preds[:, 0]
    template["G"] = bin_preds[:, 1]
    template["A"] = bin_preds[:, 2]

    # ensure output directory exists
    out_dir = "submission"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, out_csv)
    tmp_path = out_path + ".tmp"
    template.to_csv(tmp_path, index=False)
    os.replace(tmp_path, out_path)

    print(f"Kaggle submission saved to: {out_path}")


In [6]:
# show the evaluating metrics on offsite data
def evaluating_metrics(y_true, y_pred, backbone, task_name,split_name):
    
    disease_names = ["DR", "Glaucoma", "AMD"]
    rows = []
    f1_list = []

    print(f"\n{split_name.upper()} test results for {backbone} - {task_name}")

    for i, disease in enumerate(disease_names):
        yt = y_true[:, i]
        yp = y_pred[:, i]

        acc = accuracy_score(yt, yp)
        precision = precision_score(yt, yp, zero_division=0)
        recall = recall_score(yt, yp, zero_division=0)
        f1 = f1_score(yt, yp, zero_division=0)
        kappa = cohen_kappa_score(yt, yp)

        f1_list.append(f1)

        """# print in the required format (optional)
        print(f"{disease} Results [{backbone}] ({split_name})")
        print(f"Accuracy : {acc:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall   : {recall:.4f}")
        print(f"F1-score : {f1:.4f}")
        print(f"Kappa    : {kappa:.4f}")
        print("-----")"""

        rows.append({
            "Backbone": backbone,
            "Task": task_name,
            "Split": split_name,
            "Disease": disease,
            "Accuracy": acc,
            "Precision": precision,
            "Recall": recall,
            "F1-score": f1,
            "Kappa": kappa,
        })

    avg_f1 = sum(f1_list) / len(f1_list)
    print(f"Average F1 over 3 diseases ({split_name}): {avg_f1:.4f}\n")

    rows.append({
        "Backbone": backbone,
        "Task": task_name,
        "Split": split_name,
        "Disease": "Average F1",
        "Accuracy": None,
        "Precision": None,
        "Recall": None,
        "F1-score": avg_f1,
        "Kappa": None,
    })

    return pd.DataFrame(rows)

## Configuration

This section defines all dataset paths, pretrained backbone locations, and key training hyperparameters. Adjust these values to match your local setup before running the experiments. 

In [7]:
# Configuration (edit paths here)


# Labeled splits
train_csv = "train.csv"
val_csv = "val.csv"
offsite_test_csv = "offsite_test.csv"

train_img_dir = "./images/train"
val_img_dir = "./images/val"
offsite_img_dir = "./images/offsite_test"

# unlabeled onsite test (for kaggle submission)
onsite_csv = "onsite_test_submission.csv"
onsite_img_dir = "./images/onsite_test"

# optional: your own pretrained backbones
pretrained_resnet18 = "./pretrained_backbone/ckpt_resnet18_ep50.pt"
pretrained_efficient = "./pretrained_backbone/ckpt_efficientnet_ep50.pt"

img_size = 256
epochs = 20
batch_size = 32
lr = 1e-4
save_dir = "checkpoints"


## Training

This section includes all training runs used to generate the results presented in the report and submitted to Kaggle. The preliminary results and metrics shown here were used directly in the report, and the hyperparameters in the code match the exact configurations of the final experiments. Because training is stochastic, your results may not match mine exactly, but repeated runs should produce similar outcomes.


Each task is organized into its own code block. For example:

- `# Task 1.1  No fine-tuning, resnet18`  


After running multiple trials, the best-performing configuration was selected for the final model and Kaggle submission. Your single run may not match the final score exactly, but repeated runs should produce similar performance, though the number of repeated runs maybe large. And you need to have enough VRAM as some of the configurations were using batch_size = 128.


In [8]:
# Task 1.1 No fine-tuning, resnet18

backbone = "resnet18" 

if backbone == "resnet18":
    pretrained_path = pretrained_resnet18
elif backbone == "efficientnet":
    pretrained_path = pretrained_efficient
else:
    raise ValueError("unknown backbone")

#offsite

ckpt_task11, y_true_offsite, y_pred_offsite = train_one_backbone(
    backbone=backbone,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=offsite_test_csv,
    train_image_dir=train_img_dir,
    val_image_dir=val_img_dir,
    test_image_dir=offsite_img_dir,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr,
    img_size=img_size,
    save_dir=save_dir,
    pretrained_backbone=pretrained_path,
    task="no_finetune",
)




#onsite
generate_kaggle_submission(
    backbone=backbone,
    ckpt_path="./pretrained_backbone/ckpt_resnet18_ep50.pt",
    onsite_csv=onsite_csv,          
    onsite_image_dir=onsite_img_dir,
    img_size=img_size,
    batch_size=batch_size,
    out_csv=f"submission_{backbone}_task1_1.csv",
)



df_offsite = evaluating_metrics(
    y_true=y_true_offsite,
    y_pred=y_pred_offsite,
    backbone=backbone,
    task_name="no_finetune",
    split_name="offsite",
)
df_offsite

cuda
Task1.1 No fine-tuning  |  Backbone: resnet18
pos_weight: tensor([0.5474, 3.9080, 4.6338], device='cuda:0')
Loaded pretrained weights from ./pretrained_backbone/ckpt_resnet18_ep50.pt
[resnet18] Task1.1 No fine-tuning: no training, model saved at checkpoints\csu_task1_1_resnet18.pt
Kaggle submission saved to: submission\submission_resnet18_task1_1.csv

OFFSITE test results for resnet18 - no_finetune
Average F1 over 3 diseases (offsite): 0.5126



,Backbone,Task,Split,Disease,Accuracy,Precision,Recall,F1-score,Kappa
0,resnet18,no_finetune,offsite,DR,0.515,0.717172,0.507143,0.594142,0.033865
1,resnet18,no_finetune,offsite,Glaucoma,0.785,0.575000,0.469388,0.516854,0.380403
2,resnet18,no_finetune,offsite,AMD,0.785,0.301887,0.727273,0.426667,0.321124
3,resnet18,no_finetune,offsite,Average F1,NaN,NaN,NaN,0.512554,NaN


In [9]:
# Task 1.1 No fine-tuning, efficientnet

backbone = "efficientnet" 

if backbone == "resnet18":
    pretrained_path = pretrained_resnet18
elif backbone == "efficientnet":
    pretrained_path = pretrained_efficient
else:
    raise ValueError("unknown backbone")

#offsite
ckpt_task11, y_true_offsite, y_pred_offsite = train_one_backbone(
    backbone=backbone,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=offsite_test_csv,
    train_image_dir=train_img_dir,
    val_image_dir=val_img_dir,
    test_image_dir=offsite_img_dir,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr,
    img_size=img_size,
    save_dir=save_dir,
    pretrained_backbone=pretrained_path,
    task="no_finetune",
)





#onsite
generate_kaggle_submission(
    backbone=backbone,
    ckpt_path=ckpt_task11,
    onsite_csv=onsite_csv,          
    onsite_image_dir=onsite_img_dir,
    img_size=img_size,
    batch_size=batch_size,
    out_csv=f"submission_{backbone}_task1_1.csv",
)

df_offsite = evaluating_metrics(
    y_true=y_true_offsite,
    y_pred=y_pred_offsite,
    backbone=backbone,
    task_name="no_finetune",
    split_name="offsite",
)
df_offsite



cuda
Task1.1 No fine-tuning  |  Backbone: efficientnet
pos_weight: tensor([0.5474, 3.9080, 4.6338], device='cuda:0')
Loaded pretrained weights from ./pretrained_backbone/ckpt_efficientnet_ep50.pt
[efficientnet] Task1.1 No fine-tuning: no training, model saved at checkpoints\csu_task1_1_efficientnet.pt
Kaggle submission saved to: submission\submission_efficientnet_task1_1.csv

OFFSITE test results for efficientnet - no_finetune
Average F1 over 3 diseases (offsite): 0.5541



,Backbone,Task,Split,Disease,Accuracy,Precision,Recall,F1-score,Kappa
0,efficientnet,no_finetune,offsite,DR,0.600,0.745902,0.650000,0.694656,0.122807
1,efficientnet,no_finetune,offsite,Glaucoma,0.795,0.576923,0.612245,0.594059,0.457097
2,efficientnet,no_finetune,offsite,AMD,0.715,0.246377,0.772727,0.373626,0.248219
3,efficientnet,no_finetune,offsite,Average F1,NaN,NaN,NaN,0.554114,NaN


In [10]:
# Task 1.2 classfier only, resnet18

backbone = "resnet18" 

if backbone == "resnet18":
    pretrained_path = pretrained_resnet18
elif backbone == "efficientnet":
    pretrained_path = pretrained_efficient
else:
    raise ValueError("unknown backbone")

#offsite
ckpt_task12, y_true_offsite, y_pred_offsite = train_one_backbone(
    backbone=backbone,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=offsite_test_csv,
    train_image_dir=train_img_dir,
    val_image_dir=val_img_dir,
    test_image_dir=offsite_img_dir,
    epochs=30,
    batch_size=batch_size,
    lr=1e-3,
    img_size=img_size,
    save_dir=save_dir,
    pretrained_backbone=pretrained_path,
    task="cls_only",
)


#onsite
generate_kaggle_submission(
    backbone=backbone,
    ckpt_path=ckpt_task12,
    onsite_csv=onsite_csv,          
    onsite_image_dir=onsite_img_dir,
    img_size=img_size,
    batch_size=batch_size,
    out_csv=f"submission_{backbone}_task1_2.csv",
)

df_offsite = evaluating_metrics(
    y_true=y_true_offsite,
    y_pred=y_pred_offsite,
    backbone=backbone,
    task_name="cls_only",
    split_name="offsite",
)
torch.cuda.empty_cache()
df_offsite



cuda
Task1.2 Frozen backbone, classifier only  |  Backbone: resnet18
pos_weight: tensor([0.5474, 3.9080, 4.6338], device='cuda:0')
Loaded pretrained weights from ./pretrained_backbone/ckpt_resnet18_ep50.pt
[resnet18] Epoch 1/30 Train Loss: 1.4870 Val Loss: 1.2051
Saved best model for resnet18 (cls_only) at checkpoints\csu_task1_2_resnet18.pt
[resnet18] Epoch 2/30 Train Loss: 0.9486 Val Loss: 1.0159
Saved best model for resnet18 (cls_only) at checkpoints\csu_task1_2_resnet18.pt
[resnet18] Epoch 3/30 Train Loss: 0.8306 Val Loss: 0.8817
Saved best model for resnet18 (cls_only) at checkpoints\csu_task1_2_resnet18.pt
[resnet18] Epoch 4/30 Train Loss: 0.7812 Val Loss: 0.8367
Saved best model for resnet18 (cls_only) at checkpoints\csu_task1_2_resnet18.pt
[resnet18] Epoch 5/30 Train Loss: 0.7701 Val Loss: 0.8403
[resnet18] Epoch 6/30 Train Loss: 0.7545 Val Loss: 0.8191
Saved best model for resnet18 (cls_only) at checkpoints\csu_task1_2_resnet18.pt
[resnet18] Epoch 7/30 Train Loss: 0.7451 Val L

,Backbone,Task,Split,Disease,Accuracy,Precision,Recall,F1-score,Kappa
0,resnet18,cls_only,offsite,DR,0.770,0.845588,0.821429,0.833333,0.462617
1,resnet18,cls_only,offsite,Glaucoma,0.805,0.589286,0.673469,0.628571,0.497163
2,resnet18,cls_only,offsite,AMD,0.785,0.322034,0.863636,0.469136,0.367833
3,resnet18,cls_only,offsite,Average F1,NaN,NaN,NaN,0.643680,NaN


In [11]:
# Task 1.2 classfier only, efficientnet

backbone = "efficientnet"

if backbone == "resnet18":
    pretrained_path = pretrained_resnet18
elif backbone == "efficientnet":
    pretrained_path = pretrained_efficient
else:
    raise ValueError("unknown backbone")

#offsite
ckpt_task12, y_true_offsite, y_pred_offsite = train_one_backbone(
    backbone=backbone,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=offsite_test_csv,
    train_image_dir=train_img_dir,
    val_image_dir=val_img_dir,
    test_image_dir=offsite_img_dir,
    epochs=40,
    batch_size=batch_size,
    lr=2e-3,
    img_size=img_size,
    save_dir=save_dir,
    pretrained_backbone=pretrained_path,
    task="cls_only",
)


#onsite
generate_kaggle_submission(
    backbone=backbone,
    ckpt_path=ckpt_task12,
    onsite_csv=onsite_csv,          
    onsite_image_dir=onsite_img_dir,
    img_size=img_size,
    batch_size=batch_size,
    out_csv=f"submission_{backbone}_task1_2.csv",
)

df_offsite = evaluating_metrics(
    y_true=y_true_offsite,
    y_pred=y_pred_offsite,
    backbone=backbone,
    task_name="cls_only",
    split_name="offsite",
)
torch.cuda.empty_cache()
df_offsite



cuda
Task1.2 Frozen backbone, classifier only  |  Backbone: efficientnet
pos_weight: tensor([0.5474, 3.9080, 4.6338], device='cuda:0')
Loaded pretrained weights from ./pretrained_backbone/ckpt_efficientnet_ep50.pt
[efficientnet] Epoch 1/40 Train Loss: 1.0352 Val Loss: 0.9356
Saved best model for efficientnet (cls_only) at checkpoints\csu_task1_2_efficientnet.pt
[efficientnet] Epoch 2/40 Train Loss: 0.6710 Val Loss: 0.7640
Saved best model for efficientnet (cls_only) at checkpoints\csu_task1_2_efficientnet.pt
[efficientnet] Epoch 3/40 Train Loss: 0.6180 Val Loss: 0.7128
Saved best model for efficientnet (cls_only) at checkpoints\csu_task1_2_efficientnet.pt
[efficientnet] Epoch 4/40 Train Loss: 0.6111 Val Loss: 0.8110
[efficientnet] Epoch 5/40 Train Loss: 0.5995 Val Loss: 0.7528
[efficientnet] Epoch 6/40 Train Loss: 0.5676 Val Loss: 0.7472
[efficientnet] Epoch 7/40 Train Loss: 0.5559 Val Loss: 0.7321
[efficientnet] Epoch 8/40 Train Loss: 0.5301 Val Loss: 0.7646
[efficientnet] Epoch 9/40 

,Backbone,Task,Split,Disease,Accuracy,Precision,Recall,F1-score,Kappa
0,efficientnet,cls_only,offsite,DR,0.765,0.904348,0.742857,0.815686,0.500000
1,efficientnet,cls_only,offsite,Glaucoma,0.805,0.580645,0.734694,0.648649,0.516249
2,efficientnet,cls_only,offsite,AMD,0.825,0.372549,0.863636,0.520548,0.433474
3,efficientnet,cls_only,offsite,Average F1,NaN,NaN,NaN,0.661628,NaN


In [12]:
# Task 1.3 full fine-tune, resnet18

backbone = "resnet18" 

if backbone == "resnet18":
    pretrained_path = pretrained_resnet18
elif backbone == "efficientnet":
    pretrained_path = pretrained_efficient
else:
    raise ValueError("unknown backbone")

#offsite
ckpt_task13, y_true_offsite, y_pred_offsite = train_one_backbone(
    backbone=backbone,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=offsite_test_csv,
    train_image_dir=train_img_dir,
    val_image_dir=val_img_dir,
    test_image_dir=offsite_img_dir,
    epochs=20,
    batch_size=128,
    lr=2e-4,
    img_size=img_size,
    save_dir=save_dir,
    pretrained_backbone=pretrained_path,
    task="full_ft",
)


#onsite
generate_kaggle_submission(
    backbone=backbone,
    ckpt_path=ckpt_task13,
    onsite_csv=onsite_csv,          
    onsite_image_dir=onsite_img_dir,
    img_size=img_size,
    batch_size=batch_size,
    out_csv=f"submission_{backbone}_task1_3.csv",
)

df_offsite = evaluating_metrics(
    y_true=y_true_offsite,
    y_pred=y_pred_offsite,
    backbone=backbone,
    task_name="full_ft",
    split_name="offsite",
)
torch.cuda.empty_cache()
df_offsite



cuda
Task1.3 Full fine-tuning  |  Backbone: resnet18
pos_weight: tensor([0.5474, 3.9080, 4.6338], device='cuda:0')
Loaded pretrained weights from ./pretrained_backbone/ckpt_resnet18_ep50.pt
[resnet18] Epoch 1/20 Train Loss: 1.2749 Val Loss: 2.2040
Saved best model for resnet18 (full_ft) at checkpoints\csu_task1_3_resnet18.pt
[resnet18] Epoch 2/20 Train Loss: 0.4850 Val Loss: 1.2188
Saved best model for resnet18 (full_ft) at checkpoints\csu_task1_3_resnet18.pt
[resnet18] Epoch 3/20 Train Loss: 0.3119 Val Loss: 0.8805
Saved best model for resnet18 (full_ft) at checkpoints\csu_task1_3_resnet18.pt
[resnet18] Epoch 4/20 Train Loss: 0.2031 Val Loss: 0.7935
Saved best model for resnet18 (full_ft) at checkpoints\csu_task1_3_resnet18.pt
[resnet18] Epoch 5/20 Train Loss: 0.1312 Val Loss: 0.8016
[resnet18] Epoch 6/20 Train Loss: 0.0837 Val Loss: 0.8398
[resnet18] Epoch 7/20 Train Loss: 0.0546 Val Loss: 0.8304
[resnet18] Epoch 8/20 Train Loss: 0.0342 Val Loss: 0.9042
[resnet18] Epoch 9/20 Train Lo

,Backbone,Task,Split,Disease,Accuracy,Precision,Recall,F1-score,Kappa
0,resnet18,full_ft,offsite,DR,0.81,0.859155,0.871429,0.865248,0.543269
1,resnet18,full_ft,offsite,Glaucoma,0.89,0.800000,0.734694,0.765957,0.694232
2,resnet18,full_ft,offsite,AMD,0.86,0.411765,0.636364,0.500000,0.422918
3,resnet18,full_ft,offsite,Average F1,NaN,NaN,NaN,0.710402,NaN


In [25]:
# Task 1.3 full fine-tune, efficientnet

backbone = "efficientnet" 

if backbone == "resnet18":
    pretrained_path = pretrained_resnet18
elif backbone == "efficientnet":
    pretrained_path = pretrained_efficient
else:
    raise ValueError("unknown backbone")

#offsite
ckpt_task13, y_true_offsite, y_pred_offsite = train_one_backbone(
    backbone=backbone,
    train_csv=train_csv,
    val_csv=val_csv,
    test_csv=offsite_test_csv,
    train_image_dir=train_img_dir,
    val_image_dir=val_img_dir,
    test_image_dir=offsite_img_dir,
    epochs=20,
    batch_size=128,
    lr=4e-4,
    img_size=img_size,
    save_dir=save_dir,
    pretrained_backbone=pretrained_path,
    task="full_ft",
)


#onsite
generate_kaggle_submission(
    backbone=backbone,
    ckpt_path=ckpt_task13,
    onsite_csv=onsite_csv,          
    onsite_image_dir=onsite_img_dir,
    img_size=img_size,
    batch_size=batch_size,
    out_csv=f"submission_{backbone}_task1_3.csv",
)

df_offsite = evaluating_metrics(
    y_true=y_true_offsite,
    y_pred=y_pred_offsite,
    backbone=backbone,
    task_name="full_ft",
    split_name="offsite",
)


torch.cuda.empty_cache()
df_offsite

cuda
Task1.3 Full fine-tuning  |  Backbone: efficientnet
pos_weight: tensor([0.5474, 3.9080, 4.6338], device='cuda:0')
Loaded pretrained weights from ./pretrained_backbone/ckpt_efficientnet_ep50.pt
[efficientnet] Epoch 1/20 Train Loss: 1.2142 Val Loss: 1.0799
Saved best model for efficientnet (full_ft) at checkpoints\csu_task1_3_efficientnet.pt
[efficientnet] Epoch 2/20 Train Loss: 0.4626 Val Loss: 0.9317
Saved best model for efficientnet (full_ft) at checkpoints\csu_task1_3_efficientnet.pt
[efficientnet] Epoch 3/20 Train Loss: 0.2897 Val Loss: 0.9116
Saved best model for efficientnet (full_ft) at checkpoints\csu_task1_3_efficientnet.pt
[efficientnet] Epoch 4/20 Train Loss: 0.1977 Val Loss: 0.8816
Saved best model for efficientnet (full_ft) at checkpoints\csu_task1_3_efficientnet.pt
[efficientnet] Epoch 5/20 Train Loss: 0.1348 Val Loss: 1.0668
[efficientnet] Epoch 6/20 Train Loss: 0.1017 Val Loss: 1.2950
[efficientnet] Epoch 7/20 Train Loss: 0.0745 Val Loss: 1.3140
[efficientnet] Epoch

,Backbone,Task,Split,Disease,Accuracy,Precision,Recall,F1-score,Kappa
0,efficientnet,full_ft,offsite,DR,0.770,0.879032,0.778571,0.825758,0.491150
1,efficientnet,full_ft,offsite,Glaucoma,0.875,0.730769,0.775510,0.752475,0.668962
2,efficientnet,full_ft,offsite,AMD,0.865,0.424242,0.636364,0.509091,0.434437
3,efficientnet,full_ft,offsite,Average F1,NaN,NaN,NaN,0.695775,NaN
